In [26]:
from scipy.io import loadmat
from scipy.io import savemat
import numpy as np
import statistics as stat
import random
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

In [27]:
def get_signals_info(emg, restimulus, debut , fin):
    emg_signals = emg[debut:fin]
    emg_labels_ = restimulus[debut:fin]
    emg_labels = list(map(lambda x: x[0], emg_labels_))
    return emg_signals, emg_labels
    
def get_index_list(labels, index_mouve):
  index_list = ([np.where(labels == index_mouve) ])
  return np.squeeze(np.concatenate(index_list, axis=1))
  
##########################################################################################################################
def cpt_mouvements(db_labels):
  cpt = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

  for i in db_labels:
    cpt[i] += 1
  return cpt

def cpt_mouvements_v2(db_labels):
  cpt = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

  print(db_labels.shape)

  for i in range(db_labels.shape[0]):
    for j in range(db_labels.shape[1]):
      if db_labels[i][j] == 1:
          cpt[j] += 1
  return cpt

def affich_mouvements(db_labels):
  total = 0  
  cpt = cpt_mouvements_v2(db_labels)

  for j in range(len(cpt)):
    if cpt[j] > 0:
      total += cpt[j]
      print("> mouvement "+str(j)+ " : " + str(cpt[j]))
  print("> Le nombre total d'echantillons : " + str(total) )    


In [28]:
def creat_dataset(emg, restimulus, debut, fin, window = 300, window_inc = 300, ratio = (2/3)): #
  ### Signals ### 
  data_set = []
  ### labels ### 
  cpt_mouvements= 0
  select_val = int((window * ratio))  ##########################################
  labels_list = []
  current_mouv = 0
              ########################### 
  for pose in range(len(debut)) : 

    train_signals, train_labels = get_signals_info(emg, restimulus, debut[pose], fin[pose])
      
    for i in range(int((fin[pose]-debut[pose])/window_inc)):
      startL = i * window_inc
      endL = startL + window
      if endL > (fin[pose]-debut[pose]):
        break;
      else:

        ### labels ###
        for j in range(startL, endL):
          if train_labels[j] > 0:
            if train_labels[j] > current_mouv:
                current_mouv = train_labels[j]
                cpt_mouvements = 1
            elif int(train_labels[j]) == int(current_mouv):
                cpt_mouvements += 1
        if cpt_mouvements > select_val:
                if current_mouv > 9:
                   labels_list.append(current_mouv-2)
                else:
                  labels_list.append(current_mouv)

                ### Signals ###
                data_set.append(train_signals[startL:endL])           
        elif cpt_mouvements == 0:
                labels_list.append(0)
                ### Signals ###
                data_set.append(train_signals[startL:endL])
                
        cpt_mouvements = 0
        current_mouv = 0

  return data_set, labels_list

def remouve_mouves(train_data,train_labels,test_data,test_labels):

    cpt_train = cpt_mouvements(train_labels)
    cpt_test = cpt_mouvements(test_labels)


    train_mean = int(stat.mean(cpt_train[1:16]))
    test_mean = int(stat.mean(cpt_test[1:16]))

    for i in range(len(cpt_train)):

      to_remove_train = cpt_train[i] - int(train_mean)
      to_remove_test = cpt_test[i] - int(test_mean)

      if to_remove_train > 0:  
        zeros_list = get_index_list(train_labels, i)
        zeros_toremove = random.sample(list(zeros_list), to_remove_train)

        train_labels = np.delete(train_labels, zeros_toremove, axis=0)
        train_data = np.delete(train_data, zeros_toremove, axis=0)

      if to_remove_test > 0:
        zeros_list = get_index_list(test_labels, i)
        zeros_toremove = random.sample(list(zeros_list), to_remove_test)

        test_labels = np.delete(test_labels, zeros_toremove, axis=0)
        test_data = np.delete(test_data, zeros_toremove, axis=0)
    
    return train_data,train_labels,test_data,test_labels


In [29]:
def dataset_normalize(train_data, test_data):

  train_data = np.array(train_data)
  test_data = np.array(test_data)

  train_x = train_data.shape[0]
  train_y = train_data.shape[1]
  train_z = train_data.shape[2]

  test_x = test_data.shape[0]
  test_y = test_data.shape[1]
  test_z = test_data.shape[2]

  train_data = train_data.reshape(train_x*train_y, train_z)
  test_data = test_data.reshape(test_x*test_y, test_z)

  scaler = StandardScaler(with_mean=True, with_std=True).fit(train_data)
  
  train_N = scaler.transform(train_data)
  test_N = scaler.transform(test_data)

  train_N = train_N.reshape(train_x, train_y, train_z)
  test_N = test_N.reshape(test_x, test_y, test_z)
  return train_N, test_N

In [30]:
#Determiner le debut et la fin de chaque repetition pour chaque mouvement 
def get_labels_interv(rerepitition):
  rerepitition = np.squeeze(rerepitition)
  global_labels = []
  local_labels = [0]
  mouvement = 1
  cpt = 0
  for i in range(len(rerepitition)):
    if rerepitition[i] == 0:
      local_labels.append(i)
      global_labels.append(local_labels)
      cpt = 0
      break
    elif rerepitition[i] != mouvement:
      cpt += 1
      local_labels.append(i)
      mouvement = rerepitition[i]
      if mouvement == 1 and cpt == 6:
        global_labels.append(local_labels)
        local_labels = [i]                            
        cpt = 0
  return global_labels

In [31]:
#L'intervalle d'etude
def start_end_list(rerepetition= [], window= 300, rep_train = [1,2,4,6], rep_test = [3,5]):
  global_labels = get_labels_interv(rerepetition)
  train_debut = []
  train_fin = []
  test_debut = []
  test_fin = []

  for i in global_labels[:9]: 
    for j in rep_train:
      train_debut.append(i[j-1])
      fin = int((i[j]-i[j-1])/window)*window + i[j-1]
      train_fin.append(fin)
    for k in rep_test:
      test_debut.append(i[k-1])
      fin = int((i[k]-i[k-1])/window)*window + i[k-1]
      test_fin.append(fin)
    
  for i in global_labels[11:]: 
    for j in rep_train:
      train_debut.append(i[j-1])
      fin = int((i[j]-i[j-1])/window)*window + i[j-1]
      train_fin.append(fin)
    for k in rep_test:
      test_debut.append(i[k-1])
      fin = int((i[k]-i[k-1])/window)*window + i[k-1]
      test_fin.append(fin)

  return train_debut,train_fin,test_debut,test_fin

In [32]:
def creation(filename, window = 300, window_inc = 300, ratio = (2/3)):
  train_data = []
  train_labels = []
  test_data = []
  test_labels = []
  
  for sujet in filename:
    data = loadmat(sujet)
    emg = data['emg']
    restimulus = data['restimulus']
    rerepetition = data['rerepetition']

    train_debut,train_fin,test_debut,test_fin = start_end_list(rerepetition, window= window)

    trainX, trainY = creat_dataset(emg = emg, restimulus = restimulus, debut = train_debut, fin = train_fin, window = window, window_inc = window_inc, ratio = ratio )
    testX, testY = creat_dataset(emg = emg, restimulus = restimulus, debut = test_debut, fin = test_fin, window = window, window_inc = window_inc, ratio = ratio)

    train_data.extend(trainX)
    train_labels.extend(trainY)
    test_data.extend(testX)
    test_labels.extend(testY)

  ### Signals ### 
  train_data = np.array(train_data)
  test_data = np.array(test_data)
  ### labels ###      
  train_labels = np.array(train_labels)
  test_labels = np.array(test_labels)

  train_data,train_labels,test_data,test_labels = remouve_mouves(train_data,train_labels,test_data,test_labels)


  train_labels = to_categorical(train_labels) 
  test_labels = to_categorical(test_labels)

  return train_data,train_labels,test_data,test_labels

In [33]:
############# Creation du Dataset ########
Subjects = []
for i in range(1, 12):  # Looping through DB3_s1 to DB3_s11
    subject_folder = f"DB3_s{i}"  # Folder name (e.g., DB3_s1)
    subject_file = f"S{i}_E1_A1.mat"  # File name (e.g., S1_E1_A1.mat)
    
    input_path = fr"G:\former students work\Code source\DB3\{subject_folder}\{subject_file}"
    train_data, train_labels, test_data, test_labels = creation(filename=[input_path], window=300, window_inc=200)

    output_path = fr"G:\former students work\Code source\DB3_new\DB3_s{i}_E1_A1_150_100.mat"
    new_data = {'train_data': train_data, 'train_labels': train_labels, 'test_data': test_data, 'test_labels': test_labels}
    savemat(output_path, new_data)


In [34]:
def create_normalized_dataset(filename):

    data = loadmat(r""+ filename +"")
    train_data = data['train_data']
    test_data  = data['test_data']

    train_N, test_N = dataset_normalize(train_data, test_data)
    return train_N, data['train_labels'], test_N, data['test_labels']

In [35]:
############# Creation d'un dataset avec des données normalisées ########
for i in range(1, 12):  # Adjusted to match DB3_s1 to DB3_s11
    Subject = fr"G:\former students work\Code source\DB3_new\DB3_s{i}_E1_A1_150_100.mat"
    train_N, train_labels, test_N, test_labels = create_normalized_dataset(Subject)

    output_path = fr"G:\former students work\Code source\DB3_new\DB3_s{i}_E1_A1_150_100_N.mat"
    new_data = {'train_data': train_N, 'train_labels': train_labels, 'test_data': test_N, 'test_labels': test_labels}
    savemat(output_path, new_data)
